<div style="background-color:teal; color:white; padding:10px; font-size:20px">
Set Up

*** The installs below address some issue within the original environment (Image: Data Science 3.0, Kernel: Python 3) but may be unnecessary for you.

In [2]:
!conda update -n base -c defaults conda -y

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.7.3

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.3



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    appdirs-1.4.4              |     pyhd3eb1b0_0          12 KB
    asttokens-2.0.5            |     pyhd3eb1b0_0          20 KB
    backcall-0.2.0             |     pyhd3eb1b0_0          13 KB
    blas-2.118                 |         openblas          14 KB  conda-forge
    blas-devel-3.9.0           |18_linux64_openblas          14 KB  conda-forge
    brotli-1.0.9               |       h5eee18b_7          18 KB
    brotli-bin-1.0.9 

In [3]:
!conda install pytorch torchvision -c pytorch -y

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.7.3

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.3



# All requested packages already installed.



In [4]:
!pip install boto3


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import boto3
import os

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


<div style="background-color:teal; color:white; padding:10px; font-size:20px">
Data

In [6]:
train_path = "/root/Experiments_tut/toy_pytorch_project/data/cifar10_foldered_subset/train"
test_path = "/root/Experiments_tut/toy_pytorch_project/data/cifar10_foldered_subset/train"

load_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

train_set = torchvision.datasets.ImageFolder(root=train_path, transform=load_transform)
test_set = torchvision.datasets.ImageFolder(root=test_path, transform=load_transform)

<div style="background-color:teal; color:white; padding:10px; font-size:20px">
Model Definition

In [7]:
class CIFAR10Net(nn.Module):
    def __init__(self):
        super(CIFAR10Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc1 = nn.Linear(256, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn6 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 10)
        
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        
        x = self.adaptive_pool(x)
        x = x.view(-1, 256)
        
        x = F.leaky_relu(self.bn5(self.fc1(x)))
        x = self.dropout(x)
        
        x = F.leaky_relu(self.bn6(self.fc2(x)))
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x

<div style="background-color:teal; color:white; padding:10px; font-size:20px">
Data Loaders

In [8]:
trainloader = torch.utils.data.DataLoader(train_set, batch_size=64,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_set, batch_size=64,
                                         shuffle=False, num_workers=2)

<div style="background-color:teal; color:white; padding:10px; font-size:20px">
Train Loop

In [9]:
import torch.optim as optim

model = CIFAR10Net()

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 3

for epoch in range(num_epochs):
    
    # Training
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(trainloader)
    train_accuracy = 100 * correct_train / total_train
    print(f"Epoch {epoch+1}, loss: {train_loss:.4f}, accuracy: {train_accuracy:.2f}%")
    
    # Validation
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            inputs, labels = data
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_val_loss += loss.item()

            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    val_loss = running_val_loss / len(testloader)
    val_accuracy = 100 * correct_val / total_val
    print(f"Validation loss: {val_loss:.4f}, val_accuracy: {val_accuracy:.2f}%")

Epoch 1, loss: 1.8219, accuracy: 32.84%
Validation loss: 1.6754, val_accuracy: 38.60%
Epoch 2, loss: 1.4493, accuracy: 46.88%
Validation loss: 1.3376, val_accuracy: 51.42%
Epoch 3, loss: 1.2695, accuracy: 54.44%
Validation loss: 1.2887, val_accuracy: 54.64%
